In [29]:
import torch
from torchvision import transforms

import requests
try:
  from going_modular.going_modular import data_setup, engine
except:
  !git clone https://github.com/mrdbourke/pytorch-deep-learning
  !mv pytorch-deep-learning/going_modular .
  !rm -rf pytorch-deep-learning
  from going_modular.going_modular import data_setup, engine

In [30]:
# device = "cuda" if torch.cuda.is_available else "cpu"
device = "cpu"

In [31]:
def set_seeds(seed: int = 42):
  torch.manual_seed(seed)

  torch.cuda.manual_seed(seed)

In [32]:
import os
import requests
import zipfile
from pathlib import Path

def download_data(source: str,
                  destination: str,
                  remove_zip: bool = True) -> Path:

  data_path = Path("data/")

  image_path = data_path / destination

  if not image_path.is_dir():
    image_path.mkdir(exist_ok=True, parents=True)

  response = requests.get(source)

  target_file = Path(source).name
  with open(data_path/target_file, "wb") as f:
    f.write(response.content)

  with zipfile.ZipFile(data_path/target_file, "r") as zip_ref:
    zip_ref.extractall(image_path)

  if remove_zip:
    os.remove(data_path/target_file)

  return image_path

In [33]:
image_path = download_data("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           "pizza-steak-sushi")

In [34]:
from going_modular.going_modular import data_setup
from torchvision import transforms

BATCH_SIZE = 32

train_dir = image_path / "train"
test_dir = image_path / "test"

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.254, 0.255])

transformers = transforms.Compose([transforms.Resize((64, 64)),
                                  transforms.ToTensor(),
                                  normalize])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir,
    test_dir,
    transformers,
    BATCH_SIZE)

In [35]:
import torchvision

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

automatic_transforms = weights.transforms()

model = torchvision.models.efficientnet_b0(weights=weights)

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir,
    test_dir,
    automatic_transforms,
    BATCH_SIZE)

In [37]:
weights2 = torchvision.models.EfficientNet_V2_L_Weights.DEFAULT

automatic_transforms2 = weights.transforms

model2 = torchvision.models.efficientnet_v2_l(weights=weights2)

train_dataloader2, test_dataloader2, class_names2 = data_setup.create_dataloaders(
    train_dir,
    test_dir,
    automatic_transforms,
    BATCH_SIZE)

Downloading: "https://download.pytorch.org/models/efficientnet_v2_l-59c71312.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_l-59c71312.pth
100%|██████████| 455M/455M [00:04<00:00, 113MB/s]


In [38]:
try:
  from torchinfo import summary
except:
  !pip install torchinfo
  from torchinfo import summary

summary(model,
        input_size=(32,3,224,224),
        row_settings=["var_names"],
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [39]:
from torch import nn

for param in model.features.parameters():
  param.requires_grad = False

set_seeds()
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(1280,
              len(class_names)))


In [40]:
from torch import nn

for param in model2.features.parameters():
  param.requires_grad = False

set_seeds()
model2.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(1280,
              len(class_names2)))

In [41]:
try:
  from torchinfo import summary
except:
  !pip install torchinfo
  from torchinfo import summary

summary(model,
        input_size=(32,3,224,224),
        row_settings=["var_names"],
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [42]:
import os
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
def create_writer(experiment_name: str,
                  model_name: str,
                  extra: None):
  """Creates an instance of SummaryWriter"""
  timestamp = datetime.now().strftime("%Y-%m-%d")

  log_parts = [timestamp, experiment_name, model_name]

  if extra == None:
    log_parts.append(extra)

  log_dir = os.path.join(*log_parts)

  return SummaryWriter(log_dir)

In [43]:
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

from going_modular.going_modular.engine import train_step, test_step

import pandas as pd

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter = None) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]}
    For example if training for epochs=2:
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(f"Epoch: {epoch+1} | "
              f"train_loss: {train_loss:.4f} | "
              f"train_acc: {train_acc:.4f} | "
              f"test_loss: {test_loss:.4f} | "
              f"test_acc: {test_acc:.4f}")

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        if writer:

          writer.add_scalars(main_tag="Loss",
                            tag_scalar_dict={"train_loss": train_loss,
                                              "test_loss": test_loss},
                            global_step=epoch)

          writer.add_scalars(main_tag="Accuracy",
                            tag_scalar_dict={"train_acc": train_acc,
                                              "test_acc": test_acc},
                            global_step=epoch)

          writer.add_graph(model=model,
                          input_to_model=torch.randn(32,3,224,224))

    if writer:
      writer.close()

    # Return the filled results at the end of the epochs
    return results

In [46]:
epochs = 5

set_seeds()
exp1_results = train(model,
                     train_dataloader,
                     test_dataloader,
                     optimizer,
                     loss_fn,
                     epochs,
                     device,
                     create_writer("exp1",
                                   "efficent-b0",
                                   "5_epochs"))

exp2_results = train(model2,
                    train_dataloader2,
                    test_dataloader2,
                    optimizer,
                    loss_fn,
                    3,
                    device,
                    create_writer("exp2",
                                  "efficent_v2_l",
                                  "3_epochs"))


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0358 | train_acc: 0.5117 | test_loss: 0.8255 | test_acc: 0.7642
Epoch: 2 | train_loss: 0.8808 | train_acc: 0.6719 | test_loss: 0.7873 | test_acc: 0.8153
Epoch: 3 | train_loss: 0.7237 | train_acc: 0.8789 | test_loss: 0.6365 | test_acc: 0.8968
Epoch: 4 | train_loss: 0.6569 | train_acc: 0.7617 | test_loss: 0.6014 | test_acc: 0.8655
Epoch: 5 | train_loss: 0.6234 | train_acc: 0.7969 | test_loss: 0.5887 | test_acc: 0.8561


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1435 | train_acc: 0.3164 | test_loss: 1.2178 | test_acc: 0.2803
Epoch: 2 | train_loss: 1.1259 | train_acc: 0.4258 | test_loss: 1.1960 | test_acc: 0.2699
Epoch: 3 | train_loss: 1.0832 | train_acc: 0.4453 | test_loss: 1.1936 | test_acc: 0.2377


KeyboardInterrupt: 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir 2024-11-03